<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Jupyter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Dynamic systems: 8. Reactions
## Three reactants
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

----
In this notebook, we solve the continuity equation for **three reactants** with terms for:
- advection
- diffusion
- source-sink
- reaction
- decay

----
## 2D advection-diffusion-reaction equation

Check the [Fenics project](https://fenicsproject.org/pub/tutorial/html/._ftut1010.html) for the equations.

We consider the transport equations for species $A$, $B$ and $C$:
$$
\begin{array}{rcl}
\frac{\partial A}{\partial t}
+ \vec{u} \cdot \nabla A
- \nabla \cdot (D_A \nabla A) 
&=& f_A - k_1 A B - \lambda_A A \\
\frac{\partial B}{\partial t}
+ \vec{u} \cdot \nabla B
- \nabla \cdot (D_B \nabla B) 
&=& f_B - k_1 A B - \lambda_B B \\ 
\frac{\partial C}{\partial t}
+ \vec{u} \cdot \nabla C
- \nabla \cdot (D_C \nabla C)
&=& f_C + k_1 A B - \lambda_C C
\end{array}
$$

We rewrite the transport equation into a finite difference scheme with explicit time
integration, using $\frac{\partial A}{\partial t}=\frac{A(t_{i+1})-A(t_i)}{\Delta t}$:
$$
\begin{array}{rcl}
A(t_{i+1}) = A(t_i) +
\left[
- \vec{u} \cdot \nabla A
+ \nabla \cdot (D_A \nabla A)
+ f_A - k_1 A B
- \lambda_A A
\right] \Delta t \\
B(t_{i+1}) = B(t_i) +
\left[
- \vec{u} \cdot \nabla B
+ \nabla \cdot (D_B \nabla B)
+ f_B - k_1 A B
- \lambda_B B
\right] \Delta t \\
C(t_{i+1}) = C(t_i) +
\left[
- \vec{u} \cdot \nabla C
+ \nabla \cdot (D_C \nabla C)
+ f_C + k_1 A B
- \lambda_C C
\right] \Delta t
\end{array}
$$
In the square brackets, we have the **advection term**, the **diffusion term**,
a **source-sink term**, a **reaction term**, and a **decay term**.

### Reaction

The **reaction** considered is
$$
A + B \stackrel{k_1}{<->} C
$$
which can be reformulated into the differential equation:
$$
\frac{dC}{dt} = k_1 A B
$$
Here, $k_1$ [m$^3$/mol/s] is the **reaction rate**.

### Decay 

The **decay** considered is following an exponential decay model:
$$
C(t) = C_0 e^{-\lambda_C t}
$$
which is the solution of the differential equation:
$$
\frac{dC}{dt} = -\lambda_C C
$$
Here, $\lambda_C$ [1/s] is the **decay rate**.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Set parameter values

## Create mesh

In [39]:
def create_mesh(xmin,xmax,ymin,ymax,nx=11,ny=11):
    x = np.linspace(xmin,xmax,nx)
    y = np.linspace(ymin,ymax,ny)
    dx = x[1]-x[0]
    dy = y[1]-y[0]
    X,Y = np.meshgrid(x,y)
    return X,Y,dx,dy

## Initialize field values

In [75]:
def init_fields(X,Y,velx,vely,nx,ny,shake=0.):
    # concentrations A,B,C
    A = np.zeros(nx*ny).reshape(ny,nx)
    B = np.zeros(nx*ny).reshape(ny,nx)
    C = np.zeros(nx*ny).reshape(ny,nx)
    A = (1-shake)*A + shake * np.random.random((ny,nx))
    B = (1-shake)*B + shake * np.random.random((ny,nx))
    # velocity
    Vx = np.ones(nx*ny).reshape(ny,nx)
    Vy = np.ones(nx*ny).reshape(ny,nx)
    Vx = velx*Vx
    Vy = vely*Vy
    Vabs = np.sqrt(Vx**2 + Vy**2)
    return A,B,C,Vx,Vy,Vabs

In [83]:
def inject_fields(A,B):
    A[-9:-3,1:5] = 1.
    B[3:9,1:5] = 1.
    return A,B

In [77]:
def plot_fields(X,Y,A,B,C,Vx,Vy,Vabs,filename):
    fig, axs = plt.subplots(3,1,figsize=(8,10))
    
    v = np.linspace(0.0, 1.0, 11, endpoint=True)
    #plt.contour(xi, yi, zi, v, linewidths=0.5, colors='k')
    #plt.contourf(xi, yi, zi, v, cmap=plt.cm.jet)
    #x = plt.colorbar(ticks=v)

    axs[0].set_xlim([xmin,xmax])
    axs[0].set_ylim([ymin,ymax])
    CS0=axs[0].contourf(X,Y,A,v, cmap=plt.cm.jet_r)
    axs[0].quiver(X,Y,Vx/Vabs,Vy/Vabs,Vabs,alpha=0.2,width=0.002,scale=150,pivot="middle")
    axs[0].quiver(X,Y,Vx/Vabs,Vy/Vabs, alpha=0.2,edgecolor='k', facecolor='None', 
              linewidth=.5,width=0.002,scale=150,pivot="middle")
    cbar = fig.colorbar(CS0,ax=axs[0],orientation="vertical",shrink=0.9)
    cbar.ax.set_ylabel('A [mol/m$^3$]')
    
    axs[1].set_xlim([xmin,xmax])
    axs[1].set_ylim([ymin,ymax])
    CS1=axs[1].contourf(X,Y,B,v, cmap=plt.cm.jet_r)
    cbar = fig.colorbar(CS1,ax=axs[1],orientation="vertical",shrink=0.9)
    cbar.ax.set_ylabel('B [mol/m$^3$]')
    
    axs[2].set_xlim([xmin,xmax])
    axs[2].set_ylim([ymin,ymax])
    CS2=axs[2].contourf(X,Y,C,v, cmap=plt.cm.jet_r)
    cbar = fig.colorbar(CS2,ax=axs[2],orientation="vertical",shrink=0.9)
    cbar.ax.set_ylabel('C [mol/m$^3$]')
    
    axs[0].set_title('A')
    axs[1].set_title('B')
    axs[2].set_title('C')
    #axs[0].axis('off')
    #axs[1].axis('off')
    #axs[2].axis('off')
    plt.savefig('lab08/'+filename,dpi=300)
    plt.close()

## Define differential operators as finite-difference operators

In [78]:
def diffusion(M,dx,dy):
    """L = nabla^2 M"""
    L = -4*M
    L += np.roll(M, (0,-1), (0,1)) # right neighbor
    L += np.roll(M, (0,+1), (0,1)) # left neighbor
    L += np.roll(M, (-1,0), (0,1)) # top neighbor
    L += np.roll(M, (+1,0), (0,1)) # bottom neighbor
    L = L / (dx*dy)
    return L

def advection(M,Vx,Vy,dx,dy):
    """L = V*nabla M"""
    gradx  = np.roll(M, (0,-1), (0,1)) # right neighbor
    gradx -= np.roll(M, (0,+1), (0,1)) # left neighbor
    gradx  = gradx / (2*dx)
    grady  = np.roll(M, (-1,0), (0,1)) # top neighbor
    grady -= np.roll(M, (+1,0), (0,1)) # bottom neighbor
    grady  = grady / (2*dy)
    L = Vx*gradx + Vy*grady
    return L

In [79]:
def update_fields(A,B,C,Vx,Vy,DA,DB,DC,lambA,lambB,lambdaC,k1,dt,dx,dy):
    nx = A.shape[0]
    ny = A.shape[1]
    # diffusion and advection A
    laplacian = diffusion(A,dx,dy)
    gradient  = advection(A,Vx,Vy,dx,dy)
    # Now apply the update formula
    diff_A = (-gradient + DA*laplacian - lambA*A - k1*A*B) * dt
    # diffusion and advection B
    laplacian = diffusion(B,dx,dy)
    gradient  = advection(B,Vx,Vy,dx,dy)
    # Now apply the update formula
    diff_B = (-gradient + DB*laplacian - lambB*B - k1*A*B) * dt
    # diffusion and advection C
    laplacian = diffusion(C,dx,dy)
    gradient  = advection(C,Vx,Vy,dx,dy)
    # Now apply the update formula
    diff_C = (-gradient + DC*laplacian - lambC*C + k1*A*B) * dt

    A += diff_A
    B += diff_B
    C += diff_C
    #print(A.min(),A.max())

    return A,B,C

---- 
## Examples

### 1

In [87]:
xmin = 0. 
xmax = 10.
ymin = -1. 
ymax = +1.
nx   = 101
ny   = 21

DA   = 1e-3
DB   = 1e-3
DC   = 1e-3
lambA = 0.0
lambB = 0.0
lambC = 0.0
k1 = 0.6
velx = 0.1
vely = 0.
tmin = 0.
tmax = 90#0000.
dt   = 0.001
name = 'THREE-1-'

X,Y,dx,dy = create_mesh(xmin,xmax,ymin,ymax,nx,ny)
A,B,C,Vx,Vy,Vabs = init_fields(X,Y,velx,vely,nx,ny,shake=0.1)
A,B = inject_fields(A,B)
#print(A.min(),A.max(),B.min(),B.max())
plot_fields(X,Y,A,B,C,Vx,Vy,Vabs,name+f"{0:04}.png")
time = 0.
tsave = 0.
twrite = 10.
iwrite = 1
while (time < tmax):
    #print(time)
    tsave = tsave + dt
    time = time + dt
    A,B = inject_fields(A,B)
    A,B,C = update_fields(A,B,C,Vx,Vy,DA,DB,DC,lambA,lambB,lambC,k1,dt,dx,dy)
    if (tsave >= twrite):
        tsave = 0.
        print(time,tsave,twrite,A.min(),A.max(),np.sum(A))
        plot_fields(X,Y,A,B,C,Vx,Vy,Vabs,name+f"{iwrite:04}.png")
        iwrite = iwrite + 1

10.000999999999896 0.0 10.0 -0.554335184566426 1.0447286359115193 158.71741210812422
20.002000000001463 0.0 10.0 -0.5602104954419311 0.9999980801783396 199.9497126921026
30.003000000013685 0.0 10.0 -0.5697562661908349 0.9999979859681315 243.14014482562052
40.00399999999747 0.0 10.0 -0.5813954754824016 0.9999988103153077 285.7920645793532
50.00499999997416 0.0 10.0 -0.5889649855090137 0.999998820168037 326.982736679799
60.00599999995085 0.0 10.0 -0.5960869963851579 0.9999993891611013 365.78080173681224
70.00699999997022 0.0 10.0 -0.601988760064928 0.9999993598942709 402.3527696706874
80.00800000001797 0.0 10.0 -0.6081948068536139 0.999999743936996 436.5202243284127


### 2

... done